In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import sys
import os
import pandas as pd
import time
import json

project_root = ".."
sys.path.append(project_root)

from src.support_etl import *


dap = pd.read_json(r'..\recetas_scrapper\dap.jsonl', lines=True)

with open("dap_leftoff.json", 'r', encoding='utf-8') as file:
    dap_leftoff = json.load(file)

print("Translating ingredients...")
for i in range(1+dap_leftoff["Position"],len(dap)):
    ingredients = dap["ingredientes"].loc[i]
    resultados = [
        " ".join(filter(None, [str(c) if c is not None else None, u, n]))
        for n, c, u in zip(ingredients['nombre'], ingredients['cantidad'], ingredients['unidad'])
    ]
    en_ingredients = [translate_es_en(e) for e in resultados]
    # print(en_ingredients)
    
    serving = dap["raciones"].loc[i]

    print("Getting nutrients...")

    nut_info, recipe_sum = get_nutrients(en_ingredients, int(serving))
    print(dap["url"][i])
    if isinstance(nut_info, int) and nut_info != 555:
        print("Error:", nut_info) 
        break # Detenemos el proceso si hay un error que no sea el de mal parseo de la receta
    elif isinstance(nut_info, int) and nut_info == 555:
        json.dump(dap_leftoff, open("dap_leftoff.json", "w")) # Guardar la posición actual
        continue # No guardamos la receta si no se pudo parsear
    # display(nut_info)
    # print(recipe_sum)
    
    dap_leftoff["Position"] = i
    

    filtered_nut_info = nut_info[nut_info["Weight (g)"] > 0].copy() # Filtrar ingredientes con peso > 0 para evitar subir ingredientes sin peso (macros nulos)
    columns_to_normalize = filtered_nut_info.columns[2:]  # Macros

    # Normalizar los registros para que sean cada 100 g
    
    filtered_nut_info[columns_to_normalize] = (
        filtered_nut_info[columns_to_normalize]
        .div(filtered_nut_info["Weight (g)"], axis=0)
        * 100
    )

    print("Connecting to database...")
    conn = connect_db()
    recipe_id = insert_recipe(conn, dap["titulo"].loc[i], translate_es_en(dap["titulo"].loc[i]), dap["titulo"].loc[i], dap["url"].loc[i],recipe_sum.get("weight"), recipe_sum, int(serving))
    for j in filtered_nut_info.index:
        ingredient_id = get_or_create_ingredient(conn, filtered_nut_info.loc[j].get("Ingredient").lower(), filtered_nut_info.loc[j], filtered_nut_info.loc[j].get("Ingredient").lower(), translate_en_es(filtered_nut_info.loc[j].get("Ingredient")).lower())
        insert_ingredient_recipe(conn, recipe_id, ingredient_id, float(filtered_nut_info.loc[j].get("Weight (g)")))
    json.dump(dap_leftoff, open("dap_leftoff.json", "w")) # Guardar la posición actual
    time.sleep(2)



print("End")

Translating ingredients...
Getting nutrients...
Error: 555 {'error': 'low_quality'}
https://www.directoalpaladar.com/recetas-de-aperitivos/crema-yogur-aguacate-tartar-salmon-ahumado-aperitivo-elegante-para-picoteo-finde-cualquier-otra-ocasion
Getting nutrients...
Error: 555 {'error': 'low_quality'}
https://www.directoalpaladar.com/recetas-de-carnes-y-aves/falda-de-ternera-cocinada-a-baja-temperatura-con-lacado-oriental-receta
Getting nutrients...
Error: 555 {'error': 'low_quality'}
https://www.directoalpaladar.com/recetas-de-legumbres-y-verduras/mazorcas-de-maiz-a-la-plancha-receta
Getting nutrients...
Error: 555 {'error': 'low_quality'}
https://www.directoalpaladar.com/postres/como-hacer-casa-famosa-torrija-triple-fermentacion-ricard-camarena
Getting nutrients...
https://www.directoalpaladar.com/recetas-vegetarianas/patatas-de-la-abuela-receta
Connecting to database...
Getting nutrients...
https://www.directoalpaladar.com/recetas-de-legumbres-y-verduras/tarta-de-tomates-y-anchoas-rece

KeyboardInterrupt: 